In [109]:
from pymatgen import Structure
from pymatgen.analysis.structure_analyzer import OxideType
from pymatgen.analysis.structure_analyzer import oxide_type
from pymatgen.analysis.structure_analyzer import VoronoiAnalyzer
from pymatgen.analysis.structure_analyzer import VoronoiConnectivity
from pymatgen.analysis.structure_analyzer import average_coordination_number
from pymatgen.analysis.structure_analyzer import contains_peroxide
from pymatgen.analysis.structure_analyzer import get_dimensionality
from pymatgen.analysis.structure_analyzer import get_max_bond_lengths
from pymatgen.analysis.structure_analyzer import sulfide_type
import glob
import os
import pandas as pd
import numpy as np

In [110]:
def read_cifformat_file (directory):
    database = []
    database_dict = {}
    lststructure = []
    formula = []
    mpid = []
    for filepath in glob.iglob(directory + '*.cif'):
        base = os.path.basename(filepath)
        filenames = os.path.splitext(base)
        formula_id = filenames[0].split("_")
        structure = Structure.from_file(filepath)
        lststructure.append(structure)
        formula.append(formula_id[0])
        mpid.append(formula_id[1])
        #combine_data = [formula_id[0], formula_id[1],structure]
        #database.append(combine_data)
    database_dict= {"formula": formula, "id": mpid, "structure":lststructure}
    return database_dict

In [111]:
average_coordination_number(get_lststruct)

KeyError: 'K'

In [112]:
VoronoiAnalyzer().analyze_structures(get_lststruct, step_freq=10, most_frequent_polyhedra=15)

[('[0 5 4 2 0 0 0 0]', 8),
 ('[0 4 6 4 2 0 0 0]', 8),
 ('[0 5 2 9 0 0 0 0]', 4),
 ('[0 6 0 0 0 0 0 0]', 2)]

In [113]:
def get_dataframe_from_cifdir (directory):
    get_struct = read_cifformat_file (directory)
    get_lststruct = get_struct["structure"]
    dict_struct = {"formula": get_struct["formula"], "id": get_struct["id"]}
    dict_struct["oxide_type"] = [ oxide_type(structure, relative_cutoff=1.1, return_nbonds=True) for structure in get_lststruct]
    dict_struct["number of facets with i vertices"] = [ VoronoiAnalyzer().analyze(structure, n=0) for structure in get_lststruct]
    dict_struct["the solid angle of polygon between atomi and imagej of atomj"] = [ VoronoiConnectivity(structure,cutoff=10).connectivity_array for structure in get_lststruct]
    dict_struct["Voronoi Neighbors"] = [VoronoiConnectivity(structure,cutoff=10).get_connections() for structure in get_lststruct]
    dict_struct["maximum connectivity"] = [VoronoiConnectivity(structure,cutoff=10).max_connectivity for structure in get_lststruct]
    dict_struct["contains_peroxide"] = [contains_peroxide(structure, relative_cutoff=1.1) for structure in get_lststruct]
    dict_struct["dimensionality"] = [get_dimensionality(structure, max_hkl=2, el_radius_updates=None, min_slab_size=5, min_vacuum_size=5, standardize=True, bonds=None) for structure in get_lststruct]
    dict_struct["maximum bond length"] = [get_max_bond_lengths(structure, el_radius_updates=None) for structure in get_lststruct]
    dict_struct["sulfide_type"] = [sulfide_type(structure) for structure in get_lststruct]
    df = pd.DataFrame.from_dict(dict_struct)
    return df
    

In [114]:
df = get_dataframe_from_cifdir("/home/phu/Documents/testoxide/")

/home/phu/anaconda3/envs/matminer-ver2/lib/python3.6/site-packages/pymatgen/io/cif.py:1113: UserWarning: Issues encountered while parsing CIF:
  warnings.warn("Issues encountered while parsing CIF:")
/home/phu/anaconda3/envs/matminer-ver2/lib/python3.6/site-packages/pymatgen/io/cif.py:1115: UserWarning: Some fractional co-ordinates rounded to ideal values to avoid finite precision errors.
  warnings.warn(error)


In [115]:
df

formula         id       oxide_type number of facets with i vertices  \
0    Na2O2    mp-2340    (peroxide, 6)         [0, 6, 4, 8, 0, 0, 0, 0]   
1     TiO2   mvc-6590    (oxide, 16.0)         [2, 2, 2, 2, 0, 0, 0, 0]   
2    Ga2O3     mp-886    (oxide, 12.0)         [8, 2, 0, 2, 0, 0, 0, 4]   
3      ZnO    mp-2133     (oxide, 2.0)         [7, 0, 0, 3, 0, 0, 1, 0]   
4    Ga2O3    mp-1243    (oxide, 18.0)         [7, 0, 0, 4, 0, 0, 3, 0]   
5     NaO2    mp-1901  (superoxide, 4)         [6, 0, 0, 4, 2, 0, 0, 0]   
6      KO2   mp-21325  (superoxide, 8)         [0, 6, 4, 4, 0, 2, 0, 0]   
7    Al2O3    mp-1143    (oxide, 18.0)         [7, 0, 0, 4, 0, 0, 3, 0]   
8      K2O     mp-971     (oxide, 4.0)         [4, 0, 0, 6, 0, 0, 0, 0]   
9     K2O2    mp-2672    (peroxide, 8)         [0, 4, 6, 4, 2, 0, 0, 0]   
10    NaO2     mp-614  (superoxide, 8)         [8, 0, 0, 0, 0, 6, 0, 0]   
11     KO2    mp-1866  (superoxide, 4)         [0, 6, 0, 8, 0, 0, 0, 0]   
12    SiO2  mp-557723    (oxide, 24.0)         [1, 4, 1, 2, 0, 0, 0, 0]   
13    K2O2  mp-998917    (peroxide, 6)         [0, 6, 4, 8, 0, 0, 0, 0]   
14    SiO2  mp-555521   (oxide, 128.0)         [3, 2, 3, 1, 0, 0, 0, 0]   
15    SiO2    mp-6947     (oxide, 4.0)         [0, 6, 0, 0, 0, 0, 0, 0]   
16    SiO2  mp-560826    (oxide, 48.0)         [2, 4, 0, 1, 0, 0, 0, 0]   
17    SiO2    mp-6930     (oxide, 6.0)         [4, 0, 0, 4, 0, 0, 0, 0]   
18    Na2O    mp-2352     (oxide, 4.0)         [4, 0, 0, 6, 0, 0, 0, 0]   
19    TiO2    mp-2657     (oxide, 4.0)         [0, 6, 0, 0, 0, 0, 0, 0]   
20     K2O  mp-684904     (oxide, 4.0)         [0, 4, 4, 5, 0, 0, 0, 0]   
21   Ga2O3   mp-13134    (oxide, 12.0)         [2, 0, 6, 0, 0, 0, 0, 0]   
22     KO2  mp-998916  (superoxide, 8)         [8, 0, 0, 0, 0, 6, 0, 0]   
23   Al2O3    mp-1938    (oxide, 12.0)         [6, 2, 0, 1, 2, 2, 0, 1]   
24     K2O  mp-998914     (oxide, 4.0)         [0, 6, 0, 9, 0, 0, 0, 0]   

   the solid angle of polygon between atomi and imagej of atomj  \
0   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
1   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
2   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
3   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
4   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
5   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
6   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
7   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
8   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
9   [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
10  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
11  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
12  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
13  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
14  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
15  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
16  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
17  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
18  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
19  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
20  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
21  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
22  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
23  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             
24  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...             

                                    Voronoi Neighbors  \
0   [[0, 1, 3.463530647406981], [0, 2, 3.463530647...   
1   [[0, 8, 3.5020365061204615], [0, 10, 3.5020365...   
2   [[0, 0, 0.0], [0, 1, 3.652665175379464], [0, 8...   
3   [[0, 0, 0.0], [0, 2, 2.01242066321626], [0, 3,...   
4   [[0, 6, 2.8860318941